We will copy the commands from the Titanic example and apply them to a new data set (c06_data.xlsx)

In [2]:
import pandas as pd
import numpy as np
import sys
from sklearn.model_selection import train_test_split
load("~/conjecturing/sage/conjecturing.py")

/tmp/ipykernel_3436236/3289088492.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [1]:
pwd

'/lustre/home/clarson/C07_experiment'

In [3]:
inv_file = open("2024_02_20_inv.txt", "w")
prop_file = open("2024_02_20_prop.txt", "w")

Specify the number of examples to use for conjecturing and skips.

In [5]:
num_train = 10
my_skips = 0.3

Read data. Note that we're using the c07_data file, which should be in this directory. Also note that, in constrast to the Titanic csv file, we're opening a xlsx file, so first we need to install the needed pandas package

In [10]:
 pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 9.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [6]:
my_data =pd.read_excel("c07_data.xlsx",
                    index_col=int(0),
                    header=int(0)
                    )
print(my_data.shape)
my_data.head()

(98, 37)


,QID,title,questioncontent,questiontime,category,no_answer,ques_viewership,ques_care,ques_save,user,...,Length_CARE,Length_EMPATHY,Length_ENCOURAGEMENT,Length_GREET,Length_INTRO,Length_INVITE,Length_QUES,Length_REFERRAL,Length_RELIEF,Length_SELFDISC
0,100788023,看老年相亲节目，很震撼，人越来越没人情味?,\n 今天刷QQ看点，看老年相亲节目很震撼。我发现很多人相亲，...,2021-12-30,人际\n矛盾冲突\n欺骗与信任\n,1,446,1,0,貍奴,...,0.738933,11.336394,19.642294,5.621919,10.037360,0.408828,51.137257,1.786713,61.440305,38.877376
1,100788009,爱上别人和她分手，现在内疚到不能原谅自己怎么办？,\n 最近下班后常常陷入自责的情绪，觉得自己是人渣，怎么可以因...,2021-12-30,恋爱\n失恋\n恋爱经营\n,1,710,1,0,壹心理小耳朵,...,27.390583,23.651150,17.301045,4.525563,1.527955,0.014117,57.369216,0.729676,2.207771,0.337286
2,100788013,失眠半年了，入睡困难，偶尔睡着，也被噩梦惊醒？,\n 感觉身体非常疲惫，但是大脑一直处于兴奋状态，真的睡不着，...,2021-12-30,行为\n熬夜\n,8,957,4,0,羯傲不寻,...,0.201711,30.693097,13.733834,12.506360,3.385092,0.969819,145.799155,264.619853,44.332308,621.770329
3,100788013,失眠半年了，入睡困难，偶尔睡着，也被噩梦惊醒？,\n 感觉身体非常疲惫，但是大脑一直处于兴奋状态，真的睡不着，...,2021-12-30,行为\n熬夜\n,8,957,4,0,羯傲不寻,...,0.201711,30.693097,13.733834,12.506360,3.385092,0.969819,145.799155,264.619853,44.332308,621.770329
4,100788013,失眠半年了，入睡困难，偶尔睡着，也被噩梦惊醒？,\n 感觉身体非常疲惫，但是大脑一直处于兴奋状态，真的睡不着，...,2021-12-30,行为\n熬夜\n,8,957,4,0,仙仙人,...,10.001072,42.844507,16.524492,3.395771,47.373219,0.439851,183.726062,135.148537,58.362193,73.977332


Check the data types of the columns. For categorical data, make sure the type is integer or objects. Make sure the categories do not contain special characters besides numbers and "_".

In [7]:
my_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 98 entries, 0 to 97
Data columns (total 37 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   QID                   98 non-null     int64  
 1   title                 98 non-null     object 
 2   questioncontent       98 non-null     object 
 3   questiontime          98 non-null     object 
 4   category              98 non-null     object 
 5   no_answer             98 non-null     int64  
 6   ques_viewership       98 non-null     int64  
 7   ques_care             98 non-null     int64  
 8   ques_save             98 non-null     int64  
 9   user                  98 non-null     object 
 10  userID_x_x            98 non-null     int64  
 11  userID_x_y            98 non-null     int64  
 12  userID_y              98 non-null     int64  
 13  user_history_answers  98 non-null     int64  
 14  answertime            98 non-null     object 
 15  dummy_likeness        98 non-n

Pandas thinks likeness is an integer/numeric, but it is categorical. Recast it as an object.

In [8]:
my_data = my_data.astype({"dummy_likeness": object, "Common_Prediction": object})


let's see if it changed

In [9]:
my_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 98 entries, 0 to 97
Data columns (total 37 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   QID                   98 non-null     int64  
 1   title                 98 non-null     object 
 2   questioncontent       98 non-null     object 
 3   questiontime          98 non-null     object 
 4   category              98 non-null     object 
 5   no_answer             98 non-null     int64  
 6   ques_viewership       98 non-null     int64  
 7   ques_care             98 non-null     int64  
 8   ques_save             98 non-null     int64  
 9   user                  98 non-null     object 
 10  userID_x_x            98 non-null     int64  
 11  userID_x_y            98 non-null     int64  
 12  userID_y              98 non-null     int64  
 13  user_history_answers  98 non-null     int64  
 14  answertime            98 non-null     object 
 15  dummy_likeness        98 non-n

here's the code we'll modify from the Titanic example:

invariant_names=["Age", "Fare", "SibSp", "Parch"] 
categorical_names=["Survived", "Sex","Pclass", "cabin_letter", "Embarked"]
target = "Survived"

we need at least two invariants for our experiment. The "likeness" categorical variable will be our target. We'll also need at least two categorical (property) variables, and you only see 1 below. But invariant relations will be *added* to the properties list. 

In [10]:
invariant_names=["Length_ADVICE", "Length_AFFIRMATION", "Length_APPRAISAL", "Length_CARE", "Length_EMPATHY", "Length_ENCOURAGEMENT", "Length_GREET", "Length_INTRO", "Length_INVITE", "Length_QUES", "Length_REFERRAL", "Length_RELIEF", "Length_SELFDISC"]
categorical_names=["dummy_likeness", "Common_Prediction"]
target = "dummy_likeness"

Generic code starts here. Rename target column. Select columns.

In [11]:
if target in categorical_names:
    categorical_names[categorical_names.index(target)] = "TARGET"
else:
    invariant_names[invariant_names.index(target)] = "TARGET"

my_data = my_data.rename(columns={target: "TARGET"})
my_data = my_data[invariant_names + categorical_names]
my_data.columns

Index(['Length_ADVICE', 'Length_AFFIRMATION', 'Length_APPRAISAL',
       'Length_CARE', 'Length_EMPATHY', 'Length_ENCOURAGEMENT', 'Length_GREET',
       'Length_INTRO', 'Length_INVITE', 'Length_QUES', 'Length_REFERRAL',
       'Length_RELIEF', 'Length_SELFDISC', 'TARGET', 'Common_Prediction'],
      dtype='object')

Convert categorical variables to dummies. One dummy for each binary variable and one dummy for each level for variables with more than two levels.

In [12]:
property_names = []
for col in categorical_names:
    if col != "TARGET":
        unique_vals=list(my_data[col].unique())  # if nan is a level
        #unique_vals=list(my_data[col].dropna().unique())  # if nan is not a level
        if len(unique_vals)==2: # just use one level for binary features
            property_names.append(col+"_"+str(unique_vals[1]))
        elif len(unique_vals) > 2: #one property for each level.
            for level in unique_vals:
                property_names.append(col+"_"+str(level))


if "TARGET" in categorical_names:
    target_property_names = []
    unique_vals = list(my_data["TARGET"].unique()) # if nan is a level
    #unique_vals = list(my_data["TARGET"].dropna().unique()) # if nan is not a level
    if len(unique_vals)==2:
        target_property_names.append("TARGET_"+str(unique_vals[1]))
    elif len(unique_vals) > 2:
        for level in unique_vals:
            target_property_names.append("TARGET_"+str(level))
            
my_df = pd.get_dummies(my_data, 
                       columns=categorical_names,
                       dtype=np.uint8,
                       dummy_na=True,  # False is the default.  If False, use dropna() above
                       drop_first=False) # False is the default

my_df = my_df.rename(lambda col: col.replace('.0', ''), axis='columns')
my_df.head()

,Length_ADVICE,Length_AFFIRMATION,Length_APPRAISAL,Length_CARE,Length_EMPATHY,Length_ENCOURAGEMENT,Length_GREET,Length_INTRO,Length_INVITE,Length_QUES,Length_REFERRAL,Length_RELIEF,Length_SELFDISC,TARGET_0,TARGET_1,TARGET_nan,Common_Prediction_0,Common_Prediction_nan
0,105.714081,63.969067,172.289472,0.738933,11.336394,19.642294,5.621919,10.037360,0.408828,51.137257,1.786713,61.440305,38.877376,1,0,0,1,0
1,119.746628,25.449891,70.749119,27.390583,23.651150,17.301045,4.525563,1.527955,0.014117,57.369216,0.729676,2.207771,0.337286,1,0,0,1,0
2,269.074948,9.596857,549.316636,0.201711,30.693097,13.733834,12.506360,3.385092,0.969819,145.799155,264.619853,44.332308,621.770329,1,0,0,1,0
3,269.074948,9.596857,549.316636,0.201711,30.693097,13.733834,12.506360,3.385092,0.969819,145.799155,264.619853,44.332308,621.770329,1,0,0,1,0
4,351.549667,22.400532,206.256765,10.001072,42.844507,16.524492,3.395771,47.373219,0.439851,183.726062,135.148537,58.362193,73.977332,1,0,0,1,0


Define class, invariants, properties, and target properties (if applicable).

In [13]:
class Example():
    def __init__(self, name, mydf):
        self.name = name
        self.mydf = mydf
        
for i in invariant_names:
    inv = build_inv(i)
    setattr(Example,inv.__name__,inv )

for i in property_names:
    prop = build_prop(i)
    setattr(Example, prop.__name__,prop)

if "TARGET" in categorical_names:
    for i in target_property_names:
        prop = build_prop(i)
        setattr(Example, prop.__name__, prop)
else:
    target_invariant = invariant_names.index("TARGET")
print(property_names)

[]


In [ ]:
Split into training and testing data.

In [14]:
if "TARGET" in categorical_names:
    X_train, X_test = train_test_split(
        my_df.index,
        stratify=my_data["TARGET"],  # stratify on target levels
        train_size=num_train,
        random_state=12345
    )
else:
    X_train, X_test = train_test_split(
        my_df.index,
        train_size=num_train,
        random_state=12345
    )

Create examples for conjecturing.

In [15]:
train_examples = [Example(i, my_df) for i in X_train]
test_examples = [Example(i, my_df) for i in X_test]

Get lists of invariant and property functions.

In [16]:
invariants =[]
for i in invariant_names:
    invariants.append(Example.__dict__[i])
properties=[]
for i in property_names:
    properties.append(Example.__dict__[i])
target_properties=[]
if "TARGET" in categorical_names:
    for i in target_property_names:
        target_properties.append(Example.__dict__[i])


Invariant conjecturing - upper and lower bounds.

In [17]:
#define operators for expression tree to build upper bounsand lower bouns for each class
use_operators =  { '-1', '+1', '*2', '/2', '^2', '-()', '1/', 
                  'sqrt', 'ln', 'log10', 'exp', '10^', 'ceil', 
                  'floor', 'abs', '+', '*', 'max', 'min', '-', '/', '^'}

inv_conjectures = []

if "TARGET" in categorical_names:
    for value in target_property_names:
        print(value)
        target_property = Example.__dict__[value]
        my_examples = [example for example in train_examples if target_property(example) == True]
        for inv in invariants:
            sys.stdout.flush()
            inv_of_interest = invariants.index(inv)
            conjs = conjecture(my_examples, 
                               invariants, 
                               inv_of_interest, 
                               operators=use_operators, 
                               upperBound=True, 
                               time=Integer(5)
                             # ,debug=True,
                             #  verbose=True,
                               ,skips=my_skips
                              )
            convert_conjecture_names(conjs)
            inv_conjectures += conjs

            conjs = conjecture(my_examples, 
                               invariants, 
                               inv_of_interest, 
                               operators=use_operators, 
                               upperBound=False, 
                               time=Integer(5)
                              ,skips=my_skips)
            convert_conjecture_names(conjs)
            inv_conjectures += conjs
    print(len(inv_conjectures))
    if len(target_property_names) == 1:
        value = target_property_names[0]
        print(value + " False")
        target_property = Example.__dict__[value]
        my_examples = [example for example in train_examples if target_property(example) == False]
        for inv in invariants:
            sys.stdout.flush()
            inv_of_interest = invariants.index(inv)
            conjs = conjecture(my_examples, 
                               invariants, 
                               inv_of_interest, 
                               operators=use_operators, 
                               upperBound=True, 
                               time=Integer(5)
                             # ,debug=True,
                             #  verbose=True,
                               ,skips=my_skips
                              )
            convert_conjecture_names(conjs)
            inv_conjectures += conjs

            conjs = conjecture(my_examples, 
                               invariants, 
                               inv_of_interest, 
                               operators=use_operators, 
                               upperBound=False, 
                               time=Integer(5)
                              ,skips=my_skips)
            convert_conjecture_names(conjs)
            inv_conjectures += conjs
else: # target is an invariant
    my_examples = [example for example in train_examples]
    conjs = conjecture(my_examples, 
                       invariants, 
                       target_invariant, 
                       operators=use_operators, 
                       upperBound=True, 
                       time=Integer(5)
                        # ,debug=True,
                        #  verbose=True,
                        ,skips=my_skips)
    convert_conjecture_names(conjs)
    inv_conjectures += conjs
    conjs = conjecture(my_examples, 
                       invariants, 
                       target_invariant, 
                       operators=use_operators,
                       upperBound=False, 
                       time=Integer(5)
                       ,skips=my_skips)
    convert_conjecture_names(conjs)
    inv_conjectures += conjs     
print(len(inv_conjectures))  

for c in inv_conjectures:
    inv_file.write("%s\n" % c)
    inv_file.flush()
inv_file.close()


TARGET_1
52
TARGET_1 False
260



Property conjecturing - sufficient conditions for a categorical target values. For a binary target, get sufficient conditions for the positive class and necessary conditions for the negative class.

In [18]:
all_properties = ["TARGET"] + properties + inv_conjectures #"TARGET" is just a placeholder
prop_conjs = []
conditions = []
if "TARGET" in categorical_names:
    for value in target_property_names:
        print(value)
        all_properties[0] = Example.__dict__[value]
        #print(all_properties)
        these_prop_conjs = propertyBasedConjecture(objects=train_examples, # edit here 6/27/23
                                           properties = all_properties,
                                           mainProperty=0,
                                           #verbose=True,
                                           #debug=True,
                                           skips=my_skips)
        for c in these_prop_conjs: # edit here 6/27/23 just get premises once
            conditions.append(get_premise(c, myprint=False))
        prop_conjs += these_prop_conjs
    if len(target_property_names) == 1:
        print(value + " Necessary")
        all_properties[0] = Example.__dict__[value]
        these_prop_conjs = propertyBasedConjecture(objects=train_examples,  # edit here 6/27/23
                                           properties = all_properties,
                                           mainProperty=0,
                                           sufficient=False,
                                           #verbose=True,
                                            #  debug=True,
                                             skips=my_skips)
        for c in these_prop_conjs:
            conditions.append(get_conclusion(c, myprint=False))
        prop_conjs += these_prop_conjs  # edit here 6/27/23
        
for c in prop_conjs:
    prop_file.write("%s\n" % convert_name_back(c.__name__))
    prop_file.flush()
    
prop_file.close()

TARGET_1


/lustre/home/clarson/conjecturing/sage/conjecturing.py:177: RuntimeWarning: overflow encountered in scalar power
  stack.append(op(left, right))
/lustre/home/clarson/conjecturing/sage/conjecturing.py:132: RuntimeWarning: overflow encountered in exp
  stack.append(op(stack.pop()))
/lustre/home/clarson/anaconda3/envs/sage/lib/python3.11/site-packages/sage/misc/functional.py:1209: RuntimeWarning: invalid value encountered in log
  return ln(args[0], **kwds)
/lustre/home/clarson/conjecturing/sage/conjecturing.py:279: RuntimeWarning: overflow encountered in scalar power
  return (lambda x: 10**x), 1
/lustre/home/clarson/conjecturing/sage/conjecturing.py:147: RuntimeWarning: invalid value encountered in equal
  elif op == operator.pow and (right == Infinity or right == float('inf')):
/lustre/home/clarson/conjecturing/sage/conjecturing.py:154: RuntimeWarning: invalid value encountered in equal
  elif op == operator.pow and (right == -Infinity or right == float('-inf')):
/lustre/home/clarson/c

(Length_APPRAISAL_leq__minus_open_bracket_Length_EMPATHY_plus_Length_INTRO_close_bracket_times_logopen_bracket_Length_CARE_close_bracket)->(TARGET_1)
TARGET_1 Necessary
(TARGET_1)->(Length_APPRAISAL_leq__minus_open_bracket_Length_EMPATHY_plus_Length_INTRO_close_bracket_times_logopen_bracket_Length_CARE_close_bracket)


How many property conjectures are there?

In [19]:
len(prop_conjs)

2

Let's look at one of these property conjectures. Name the 0th one "conj0" and print it.

In [27]:
conj0=prop_conjs[0]
print(conj0)

(Length_APPRAISAL_leq__minus_open_bracket_Length_EMPATHY_plus_Length_INTRO_close_bracket_times_logopen_bracket_Length_CARE_close_bracket)->(TARGET_1)


The CONJECTURING program uses names that can't have special characters. So they are converted to a harder-to-read format. They can be converted back. Convert the string associated to conj0 to a more readable form.

In [22]:
convert_name_back(conj0.__name__)

'(Length_APPRAISAL<=-(Length_EMPATHY+Length_INTRO)*log(Length_CARE))->(TARGET_1)'

Let's test this conjecture using the test examples. How many are there?

In [23]:
len(test_examples)

88

Let's give the 0th test example the name example0

In [24]:
example0 = test_examples[0]

Let's see if conjecture "conj0" is true for example "exxample0" using the "evaluate" method for CONJECTURE objects.

In [25]:
conj0.evaluate(example0)

True

Now let's see how many of the 88 test examples are true of conjecture "conj0"

In [26]:
count = 0
for example in test_examples:
    if conj0.evaluate(example) == True:
        count = count + 1
print(count)        

84


What does this mean?